<a href="https://colab.research.google.com/github/yukinaga/twitter_bot/blob/master/section_4/02_reply_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# seq2seqモデルの訓練
対話文のデータセットを使って、Seq2Seqのモデルを訓練します。

## ライブラリののインストール
分かち書きのためにjanomeを、テキストデータの前処理のためにtorchtextをインストールします。

In [ ]:
!pip install janome==0.4.1
!pip install -U torchtext==0.7.0

## Google ドライブとの連携  
以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

## 対話文の取得
Googleドライブから、対話文のデータを取り出してリストに格納します。  



In [ ]:
import torchtext
from janome.tokenizer import Tokenizer

path = "/content/drive/My Drive/live_ai_data/"  # 保存場所を指定

j_tk = Tokenizer()
def tokenizer(text): 
    return [tok for tok in j_tk.tokenize(text, wakati=True)]  # 内包表記
 
# データセットの列を定義
input_field = torchtext.data.Field(
    sequential=True,  # データ長さが可変かどうか
    tokenize=tokenizer,  # 前処理や単語分割などのための関数
    batch_first=True,  # バッチの次元を先頭に
    lower=True  # アルファベットを小文字に変換
    )

reply_field = torchtext.data.Field(
    sequential=True,  # データ長さが可変かどうか
    tokenize=tokenizer,  # 前処理や単語分割などのための関数
    init_token = "<sos>",
    eos_token = "<eos>",
    batch_first=True,  # バッチの次元を先頭に
    lower=True  # アルファベットを小文字に変換
    )
 
# csvファイルからデータセットを作成
train_data, test_data = torchtext.data.TabularDataset.splits(
    path=path,
    train="dialogues_train.csv",
    validation="dialogues_test.csv",
    format="csv",
    fields=[("inp_text", input_field), ("rep_text", reply_field)]  # 列の設定
    )

## データセットの内容を表示
データセットの内容を一部表示します。

In [ ]:
for example in train_data.examples[:10]:
    print(example.inp_text, example.rep_text)

## 単語とインデックスの対応
単語にインデックスを割り振り、辞書として格納します。

In [ ]:
from torchtext.vocab import FastText

input_field.build_vocab(
    train_data,
    min_freq=2,
    )
reply_field.build_vocab(
    train_data,
    min_freq=2,
    )

In [ ]:
print(input_field.vocab.freqs)  # 各単語の出現頻度
print()
print(input_field.vocab.stoi)  # 　キーが単語 、値がインデックスの辞書
print()
print(input_field.vocab.itos)  # 　単語がインデックス順に格納されたリスト

## Iteratorの設定
バッチごとに学習を行うために、Iteratorを設定します。  

In [ ]:
# Iteratorの設定
batch_size = 32

train_iterator = torchtext.data.Iterator(
    train_data,
    batch_size=batch_size, 
    train=True  # シャッフルして取り出す
)

test_iterator = torchtext.data.Iterator(
    test_data,
    batch_size=batch_size, 
    train=False,
    sort=False
)

ミニバッチを取り出して、内容を表示します。  
ミニバッチには、単語をインデックスに置き換えた文章が格納されます。



In [ ]:
batch = next(iter(train_iterator))  # ミニバッチを取り出す
print(batch.inp_text.size())  # ミニバッチにおける入力のサイズ
print(batch.inp_text[0])  # 最初の要素
print(batch.rep_text.size())  # ミニバッチにおける応答のサイズ
print(batch.rep_text[0])  # 最初の要素

## Encoderのクラス
Encoderをクラスとして実装します。  
RNN部分にはGRUを使用します。  

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, n_h, n_vocab, n_emb):
        super().__init__()
        
        self.n_h = n_h

        # 埋め込み層
        self.embedding = nn.Embedding(n_vocab, n_emb)

        self.gru = nn.GRU(  # GRU層
            input_size=n_emb,  # 入力サイズ
            hidden_size=n_h,  # ニューロン数
            batch_first=True,  # 入力を (バッチサイズ, 時系列の数, 入力の数) にする
        )

    def forward(self, x):
        y = self.embedding(x)  # 単語をベクトルに変換
        y, h = self.gru(y)
        return y, h

## Decoderのクラス
Decoderをクラスとして実装します。  
RNN部分にはGRUを使用します。  
GRU層の出力は、全結合層を経てDecoderの出力となります。

In [ ]:
class Decoder(nn.Module):
    def __init__(self, n_h, n_out, n_vocab, n_emb):
        super().__init__()
        
        self.n_h = n_h
        self.n_out = n_out

        # 埋め込み層
        self.embedding = nn.Embedding(n_vocab, n_emb)

        self.gru = nn.GRU(  # GRU層
            input_size=n_emb,  # 入力サイズ
            hidden_size=n_h,  # ニューロン数
            batch_first=True,  # 入力を (バッチサイズ, 時系列の数, 入力の数) にする
        )

        self.fc = nn.Linear(n_h, n_out)
                
    def forward(self, x, h_encoder):
        y = self.embedding(x)  # 単語をベクトルに変換
        y, h = self.gru(y, h_encoder)
        y = self.fc(y)
        y = F.softmax(y, dim=2)
        return y, h

## Seq2Seqのクラス
Seq2Seqを構築します。  
`is_gpu`が`True`であれば、GPU対応を行います。

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, is_gpu=True):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.is_gpu = is_gpu
        if self.is_gpu:
            self.encoder.cuda()
            self.decoder.cuda()
        
    def forward(self, x_encoder, x_decoder):  # 訓練に使用
        if self.is_gpu:
            x_encoder, x_decoder = x_encoder.cuda(), x_decoder.cuda()

        batch_size = x_decoder.shape[0]
        n_time = x_decoder.shape[1]
        y, h = self.encoder(x_encoder)

        y_decoder = torch.zeros(batch_size, n_time, self.decoder.n_out)
        if self.is_gpu:
            y_decoder = y_decoder.cuda()

        for t in range(0, n_time):
            x = x_decoder[:, t:t+1]  # Decoderの入力を使用
            y, h= self.decoder(x, h)
            y_decoder[:, t:t+1, :] = y  
        return y_decoder

    def predict(self, x_encoder):  # 予測に使用
        if self.is_gpu:
            x_encoder = x_encoder.cuda()

        batch_size = x_encoder.shape[0]
        n_time = x_encoder.shape[1]
        y, h = self.encoder(x_encoder)

        y_decoder = torch.zeros(batch_size, n_time, dtype=torch.long)
        if self.is_gpu:
            y_decoder = y_decoder.cuda() 

        y = torch.ones(batch_size, 1, dtype=torch.long) * input_field.vocab.stoi["<sos>"]
        for t in range(0, n_time):
            x = y  # 前の時刻の出力を入力に
            if self.is_gpu:
                x = x.cuda()
            y, h= self.decoder(x, h)
            y = y.argmax(2)
            y_decoder[:, t:t+1] = y  
        return y_decoder

## 学習
DataLoaderを使ってミニバッチを取り出し、Seq2Seqのモデルを訓練します。  


In [ ]:
from torch import optim

is_gpu = True  # GPUを使用するかどうか
n_h = 512
n_vocab = len(reply_field.vocab.stoi)
n_emb = 300
n_out = n_vocab

# Seq2Seqのモデルを構築
encoder = Encoder(n_h, n_vocab, n_emb)
decoder = Decoder(n_h, n_out, n_vocab, n_emb)
seq2seq = Seq2Seq(encoder, decoder, is_gpu=is_gpu)

# 誤差関数
loss_fnc = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.Adam(seq2seq.parameters(), lr=0.001)

# 損失のログ
record_loss_train = []
record_loss_test = []

# 学習
for i in range(100):
    # 訓練モード
    seq2seq.train()

    loss_train = 0
    for j, batch in enumerate(train_iterator):
        inp, rep = batch.inp_text, batch.rep_text
        x_enc = inp
        x_dec = torch.ones(rep.size(), dtype=torch.long) * reply_field.vocab.stoi["<sos>"]
        x_dec[:, 1:] = rep[:, :-1]
        y_dec = seq2seq(x_enc, x_dec, )

        t_dec = rep.cuda() if is_gpu else rep
        loss = loss_fnc(
            y_dec.view(-1, y_dec.size()[2]),
            t_dec.view(-1)
            )
        loss_train += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if j%100==0:
            print("batch:", str(j)+"/"+str(len(train_data)//batch_size+1), "loss:", loss.item())
    loss_train /= j+1
    record_loss_train.append(loss_train)

    # 評価モード
    seq2seq.eval()

    loss_test = 0
    for j, batch in enumerate(test_iterator):
        inp, rep = batch.inp_text, batch.rep_text
        x_enc = inp
        x_dec = torch.ones(rep.size(), dtype=torch.long) * reply_field.vocab.stoi["<sos>"]
        x_dec[:, 1:] = rep[:, :-1]
        y_dec = seq2seq(x_enc, x_dec)

        t_dec = rep.cuda() if is_gpu else rep
        loss = loss_fnc(
            y_dec.view(-1, y_dec.size()[2]),
            t_dec.view(-1)
            )
        loss_test += loss.item()
    loss_test /= j+1
    record_loss_test.append(loss_test)

    if i%1 == 0:
        print("Epoch:", i, "Loss_Train:", loss_train, "Loss_Test:", loss_test)

## 誤差の推移
誤差の推移をグラフ表示します。  

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(record_loss_train)), record_loss_train, label="Train")
plt.plot(range(len(record_loss_test)), record_loss_test, label="Test")
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

## 訓練済みのモデルを使用
訓練済みのモデルを使用して、応答文を生成します。 


In [ ]:
seq2seq.eval()  # 評価モード

batch = next(iter(test_iterator))
x = batch.inp_text
y = seq2seq.predict(x)
print(y[0])
for i in range(x.size()[0]):
    inp_text = ""
    for j in range(x.size()[1]):
        inp_text += input_field.vocab.itos[x[i][j]]

    rep_text = ""
    for j in range(y.size()[1]):
        rep_text += reply_field.vocab.itos[y[i][j]]

    print("input:", inp_text)
    print("reply:", rep_text)
    print()

## 問題点

*   Encoderへの入力には、バッチ内で要素数を揃えるために多数のパディング`<pad>`が末尾に含まれている。
*   誤差の計算に、文章の終了`<eos>`と、その後の`<pad>`が使われている。

